### Importing necessary libs

In [1]:
import os
import string
from nltk import tokenize
from gensim.models import Word2Vec
from nltk import word_tokenize
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import pandas as pd
import gensim
from nltk.corpus import stopwords
import sentence_transformers
from sentence_transformers import SentenceTransformer,util
from sklearn.cluster import KMeans
import re
import statistics
from keras.layers import Average

### defining the model with the pretrainded data

In [2]:
model = SentenceTransformer('all-mpnet-base-v2', device="cuda")
# embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')


### structring words

In [3]:
listStructureWords =  []

df = pd.read_excel("structureWords.xlsx")
listStructureWords = df.iloc[:,0].tolist()

### Adding only 100 Essays for embeddings

In [44]:
#stopWords = ["directed toward young children", "from two to five","two to five", "to young children", "directed toward"]
regex = r"\b(?:{})\b".format("|".join(listStructureWords))
#regex2 = r"\b(?:{})\b".format("|".join(listStructureWords))
sentences = []
j=0
df_fileName = pd.DataFrame({})
for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\Only 100"):
   with open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\Only 100", filename)) as f:

       text = f.read()
       text = text.replace("ï»¿","")
       # for st in stopWords:
       #     text = text.replace(st,"")
       # for w in tokenize.word_tokenize(text):
       #     if w in stopWords:
       #         text = text.replace(w,"");
       #sents = tokenize.sent_tokenize(text)
       #text.translate(str.maketrans(' ', ' ', string.punctuation))
       sents = re.split(regex, text)
       sents = tokenize.sent_tokenize(text)
       #sents = text.split("\n\n")
       for s in sents:
           # if (para.isspace()):
           #     continue
               #print("The string is empty")
           #print(para)
           #print("###############")
           sentses = tokenize.sent_tokenize(s)
           if (s.isspace() or len(s) ==0):
               continue
           s = s.lower()
           #s = s.translate(str.maketrans(' ', ' ', string.punctuation))
           if (len(s.split()) <5):
               continue
           for ss in sentses:
                 if (len(ss.split()) >5):
                     sentences.append(ss)

           df_fileName = df_fileName.append({
             "ios" : j,
             "file name": filename,
              "argument": s
              },ignore_index=True)
           j = j+1


#df_fileName.to_csv("FileName with the senteces.csv")

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_31732\1340391050.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_fileName = df_fileName.append({
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_31732\1340391050.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_fileName = df_fileName.append({
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_31732\1340391050.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_fileName = df_fileName.append({
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_31732\1340391050.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_fileName = df_fileName.append({
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_3173

### Getting all the paragraphs inside the essays(Bad method)

In [ ]:
#indexs =0
paragraph_num=[]
for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2"):
    y = open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2", filename), 'r')
    paragraph = 0
    lines = y.readlines()
    for idx, line in enumerate(lines):
        if not line == '\n':
            m = re.search(r'\w', line)
            if m is None:
                continue
            str = m.group(0)
        try:
            if line == '\n' and str in lines[idx-1]:
                paragraph +=1
        except:
            pass
    if lines[-1] != '\n': # if the last line is not a new line, count a paragraph.
        paragraph +=1
    #indexs = indexs+1
    #print("The append starting" ,paragraph, indexs)
    #print("############################")
    if (paragraph >2):
        paragraph_num.append(paragraph)

average = statistics.mean(paragraph_num)
#print(indexs)
print(average)
print(round(average))
print(len(paragraph_num))

### Saving to CSV file

In [45]:
df_fileName.to_csv("FileName with the senteces.csv")

### Getting the embeddings

In [46]:
corpus_embeddings = model.encode(sentences, show_progress_bar =True, device="cuda")

corpus_embeddings.shape

Batches:   0%|          | 0/45 [00:00<?, ?it/s]

(1429, 768)

### Testing torch with cuda

In [39]:
def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

### applying the clustering method

In [47]:
k = 18

clustering, cluster_labels = mbkmeans_clusters(X=corpus_embeddings, k=k, print_silhouette_values=True)

df_clusters = pd.DataFrame({
    "text": sentences,
    "cluster": cluster_labels
})

For n_clusters = 18
Silhouette coefficient: 0.02
Inertia:715.8192749023438
Silhouette values:
    Cluster 12: Size:132 | Avg:0.18 | Min:0.09 | Max: 0.28
    Cluster 1: Size:137 | Avg:0.04 | Min:-0.13 | Max: 0.11
    Cluster 5: Size:70 | Avg:0.03 | Min:-0.04 | Max: 0.10
    Cluster 13: Size:202 | Avg:0.02 | Min:-0.10 | Max: 0.11
    Cluster 9: Size:49 | Avg:0.00 | Min:-0.15 | Max: 0.08
    Cluster 8: Size:127 | Avg:0.00 | Min:-0.11 | Max: 0.08
    Cluster 0: Size:1 | Avg:0.00 | Min:0.00 | Max: 0.00
    Cluster 10: Size:1 | Avg:0.00 | Min:0.00 | Max: 0.00
    Cluster 16: Size:96 | Avg:-0.00 | Min:-0.18 | Max: 0.07
    Cluster 3: Size:77 | Avg:-0.00 | Min:-0.08 | Max: 0.08
    Cluster 4: Size:45 | Avg:-0.01 | Min:-0.10 | Max: 0.08
    Cluster 7: Size:101 | Avg:-0.01 | Min:-0.23 | Max: 0.08
    Cluster 15: Size:69 | Avg:-0.01 | Min:-0.10 | Max: 0.07
    Cluster 2: Size:83 | Avg:-0.01 | Min:-0.12 | Max: 0.06
    Cluster 14: Size:70 | Avg:-0.01 | Min:-0.14 | Max: 0.08
    Cluster 17: Size:76

### random statments from cluster

In [48]:
test_cluster = 0
queryCount = len(df_clusters.query(f"cluster == {test_cluster}"))

print(queryCount)

if (queryCount > 10):
    for i,t in enumerate(df_clusters.query(f"cluster == {test_cluster}").sample(10).iterrows()):
        print(t[1]["text"])
        print("-------------")

else:
    for i,t in enumerate(df_clusters.query(f"cluster == {test_cluster}").sample(queryCount).iterrows()):
        print(t[1]["text"])
        print("-------------")


1
Did young children bacame addicted to modern technic?
-------------


### Most representative clusters

In [49]:
test_cluster = 0
most_representative_docs = np.argsort(
    np.linalg.norm(corpus_embeddings - clustering.cluster_centers_[test_cluster], axis=1)
)
# print(most_representative_docs[0])

if (queryCount > 10):
    for d in most_representative_docs[:10]:
        print(d)
        print(sentences[d])
        print("-------------")
else:
    for d in most_representative_docs[:queryCount]:
        print(d)
        print(sentences[d])
        print("-------------")


163
Did young children bacame addicted to modern technic?
-------------


### Saving to .csv File

In [50]:
df_clusters.to_csv(f"SbertClustering-All-Essays- {k} -Clusters.csv")

### saving most representative to csv file

In [52]:
df_mostRepresentative = pd.DataFrame({})
df_ClsuteringSentencesCount = pd.DataFrame({})

for i in range (k) :
    listSentencesMostRep =[]
    most_representative_docs_Save = np.argsort(
    np.linalg.norm(corpus_embeddings - clustering.cluster_centers_[i], axis=1)
    )
    queryMostCount = len(df_clusters.query(f"cluster == {i}"))
    df_ClsuteringSentencesCount = df_ClsuteringSentencesCount.append({
       "cluster" : i,
        "Sentences Count": queryMostCount
    },ignore_index=True)
    if (queryMostCount > 5):
        for d in most_representative_docs_Save[:5]:
            listSentencesMostRep.append(sentences[d])
    else:
        for d in most_representative_docs_Save[:queryMostCount]:
            listSentencesMostRep.append(sentences[d])

    for sent in listSentencesMostRep:
        df_mostRepresentative = df_mostRepresentative.append({
         "text": sent,
         "cluster": i
         }, ignore_index=True)



df_mostRepresentative.to_csv(f"SbertClustering-All-Essays- {k} -Clusters - 10MostRepresentative.csv")
df_ClsuteringSentencesCount.to_csv(f"SbertClustering-All-Essays- {k} - Clusters - ClusteringCount.csv")

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_31732\605274441.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ClsuteringSentencesCount = df_ClsuteringSentencesCount.append({
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_31732\605274441.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_mostRepresentative = df_mostRepresentative.append({
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_31732\605274441.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ClsuteringSentencesCount = df_ClsuteringSentencesCount.append({
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_31732\605274441.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_

### Test

In [ ]:
listTest = ["Ahmed" , "mido", "test test"]
listNumbers = [1 , 10, 15]
df_mostRepresentative = pd.DataFrame({
})

df_mostRepresentative = df_mostRepresentative.append({
    "text": listTest,
    "cluster": listNumbers
}, ignore_index=True)

listTest = ["After" , "after mido", "test after"]
listNumbers = [2 , 3, 65]

df_mostRepresentative = df_mostRepresentative.append({
    "text": listTest,
    "cluster": listNumbers
}, ignore_index=True)

df_mostRepresentative.to_csv('test.csv')

### Testing torch with cuda


In [39]:
import torch

torch.cuda.is_available()

True

### Testing word_embeddings

In [ ]:
corpus = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'A man is eating pasta.',
          'The girl is carrying a baby.',
          'The baby is carried by the woman',
          'A man is riding a horse.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'Someone in a gorilla costume is playing a set of drums.',
          'A cheetah is running behind its prey.',
          'A cheetah chases prey on across a field.'
          ]
corpus_embeddings = model.encode(corpus)

########################## TESTING #########################################################
#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(corpus_embeddings, corpus_embeddings)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-5 most similar pairs:")
for score, i, j in all_sentence_combinations[0:5]:
    print("{} \t {} \t {:.4f}".format(corpus[i], corpus[j], cos_sim[i][j]))

for sentence, embedding in zip(corpus, corpus_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

### Interensic Evaluating the clusters using pairs method

In [14]:
tpCount= 0
tnCount  = 0

fnCount = 0
fpcount = 0


df = pd.read_excel(r"D:\USERS-Load\PycharmProjects\pythonProject\CVS Files\01.08.22\annotation\GOLD vs Clusters.xlsx")

iteration = df.count()['ID']

count = 0;
for i in range(0,iteration):
    for j in range (i+1, iteration):


        clusterSystem1 = df.query(f"ID == {i}")['cluster'].iloc[0]
        clusterGold1 = df.query(f"ID == {i}")['GOLD standards'].iloc[0]
        clusterSystem2 = df.query(f"ID == {j}")['cluster'].iloc[0]
        clusterGold2 = df.query(f"ID == {j}")['GOLD standards'].iloc[0]


        if (clusterSystem1 == clusterSystem2):

            if (clusterGold1 ==clusterGold2): # it is a true positive
                tpCount = tpCount+ 1

            else:
                fnCount = fnCount + 1   # it is a false negative
        else:
            if (clusterGold1 !=clusterGold2):
                tnCount = tnCount + 1  # it is a true negative
            else:
                fpcount = fpcount+1   # it is a false positive.

print("the value of tp is", tpCount)
print("the value of tn is", tnCount)
print("the value of fn is", fnCount)
print("the value of fb is", fpcount)

print(count)


the value of tp is 86
the value of tn is 2955
the value of fn is 74
the value of fb is 206
0


### Accuracy calculation

In [15]:
accuracy = (tpCount + tnCount)/(tpCount +tnCount + fpcount+ fnCount)

print("The accuracy is ",accuracy*100)  # the value is higher than the other accuracy.

percision = tpCount / (tpCount + fpcount)

print("The percision is ",percision*100)

recall = tpCount / (tpCount + fnCount)

print("The recall is ",recall*100)

The accuracy is  91.56880457693465
The percision is  29.45205479452055
The recall is  53.75


### Calculating the purity of the clustering